<a href="https://colab.research.google.com/github/babaroscopy/colab/blob/main/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pprint
from functools import reduce

In [ ]:
class Classifier():
    data = None
    class_attr = None
    priori = {}
    cond_prob = {}
    hp = None


    def __init__(self,filename=None, class_attr=None ):
        self.data = pd.read_csv(filename, sep=',', header =(0))
        self.class_attr = class_attr

    '''
        probability(class) =    How many  times it appears in cloumn
                             __________________________________________
                                  count of all class attribute
    '''
    def calculate_priori(self):
        class_values = list(set(self.data[self.class_attr]))
        class_data =  list(self.data[self.class_attr])
        for i in class_values:
            self.priori[i]  = class_data.count(i)/float(len(class_data))
        print ("Priori Values: ", self.priori)

    '''
        Here we calculate the individual probabilites 
        P(outcome|evidence) =   P(Likelihood of Evidence) x Prior prob of outcome
                               ___________________________________________
                                                    P(Evidence)
    '''
    def get_cond_prob(self, attr, attr_type, class_value):
        data_attr = list(self.data[attr])
        class_data = list(self.data[self.class_attr])
        total =1
        for i in range(0, len(data_attr)):
            if class_data[i] == class_value and data_attr[i] == attr_type:
                total+=1
        return total/float(class_data.count(class_value))

    '''
        Here we calculate Likelihood of Evidence and multiple all individual probabilities with priori
        (Outcome|Multiple Evidence) = P(Evidence1|Outcome) x P(Evidence2|outcome) x ... x P(EvidenceN|outcome) x P(Outcome)
        scaled by P(Multiple Evidence)
    '''
    def calculate_conditional_probabilities(self, hp):
        for i in self.priori:
            self.cond_prob[i] = {}
            for j in hp:
                self.cond_prob[i].update({ hp[j]: self.get_cond_prob(j, hp[j], i)})
        print ("\nCalculated Conditional Probabilities: \n")
        pprint.pprint(self.cond_prob)

    def classify(self):
        print ("Result: ")
        for i in self.cond_prob:
            print (i, " ==> ", reduce(lambda x, y: x*y, self.cond_prob[i].values())*self.priori[i])


In [ ]:
if __name__ == "__main__":
    c = Classifier(filename="tennis.csv", class_attr="Play Tennis" )
    c.calculate_priori()
    c.hp = {"Outlook":'Rainy', "Temperature":"Mild", "Humidity":'Normal' , "Wind":'t'}

    c.calculate_conditional_probabilities(c.hp)
    c.classify()

Priori Values:  {'No': 0.35714285714285715, 'Yes': 0.6428571428571429}

Calculated Conditional Probabilities: 

{'No': {'Mild': 0.6, 'Normal': 0.4, 'Rainy': 0.2, 't': 0.2},
 'Yes': {'Mild': 0.5555555555555556,
         'Normal': 0.7777777777777778,
         'Rainy': 0.1111111111111111,
         't': 0.1111111111111111}}
Result: 
No  ==>  0.003428571428571429
Yes  ==>  0.003429355281207133
